# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os 
path = '/content/미용실_광고_이미지_유정'
os.mkdir(path)
path2 = '/content/dataset'
os.mkdir(path2)

In [ ]:
%cd /content
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/label_0.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/label_1.zip"
#!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/aug/label_0.zip"
#!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/aug/label_1.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/add_ads.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/이대미용실추가.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/직접 라벨링한 네이버지도 데이터/이미지/미용실_비광고_이미지_성우.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/직접 라벨링한 네이버지도 데이터/이미지/미용실_비광고_이미지_은비.zip"
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/직접 라벨링한 네이버지도 데이터/이미지/미용실_광고_이미지_청수.zip"
%cd /content/dataset
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/flip.zip"
%cd /content/미용실_광고_이미지_유정
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/직접 라벨링한 네이버지도 데이터/이미지/미용실_광고_이미지_유정_수정.zip"
%cd /content

/content
/content/dataset
/content/미용실_광고_이미지_유정
/content


In [ ]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [ ]:
print(get_files_count('/content/label_0')) #1017 + 1?
print(get_files_count('/content/label_1')) #1269 + 1?
print(get_files_count('/content/add_ads'))
print(get_files_count('/content/dataset/flip'))
print(get_files_count('/content/이대미용실추가'))
print(get_files_count('/content/성우_images'))
print(get_files_count('/content/미용실_비광고_이미지_은비'))
print(get_files_count('/content/미용실_광고_이미지_청수'))
print(get_files_count('/content/미용실_광고_이미지_유정'))

1018
1270
95
237
145
50
53
51
50


In [ ]:
# 라이브러리 세팅

import random
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A #증강
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split

In [ ]:
# GPU 확인
print(torch.cuda.is_available())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [ ]:
# EfficientNet 오류 안나게..
torch.backends.cudnn.enabled = False

# Hyerparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':456,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':42
}

![image.png](attachment:96771729-ea24-47ec-b706-a48a6b59384f.png)

# Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

# Data Load

In [ ]:
#train=pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/final_train.csv')
train=pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/new_train.csv')
#valid=pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/final_valid.csv')
valid=pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/new_valid.csv')

In [ ]:
train.head()

,id,img_path,label
0,82,./이대미용실추가/82.jpg,1
1,452,./label_0/452.jpg,0
2,701,./label_1/701.jpg,1
3,486,./label_1/486.jpg,1
4,709,./label_0/709.jpg,0


In [ ]:
#train.reset_index(inplace=True)
#train.head()

,index,img_path,label
0,0,./label_1/1.jpg,1
1,1,./label_1/2.jpg,1
2,2,./label_1/3.jpg,1
3,3,./label_1/4.jpg,1
4,4,./label_1/5.jpg,1


# Train-Valid Split

In [ ]:
#train = pd.concat([train,train_test], axis=0)
#train.reset_index(inplace=True, drop=True)
#train

,id,img_path,label
0,837,./label_1/837.jpg,1
1,73,./label_0/73.jpg,0
2,351,./label_0/351.jpg,0
3,1066,./label_1/1066.jpg,1
4,531,./label_0/531.jpg,0
...,...,...,...
1799,728,./label_1/728.jpg,1
1800,61,./label_1/61.jpg,1
1801,217,./label_0/217.jpg,0
1802,1024,./label_1/1024.jpg,1


In [ ]:
# 원핫 인코딩 처럼 
'''
이거 하지 말아야 
'''

#train_onehot = pd.get_dummies(train['label'])
#valid_onehot = pd.get_dummies(valid['label'])

#train = pd.concat([train, train_onehot], axis=1)
#valid = pd.concat([valid, valid_onehot], axis=1)

In [ ]:
'''
하지 말아야 
'''
#train.drop(['label'],axis=1, inplace=True)
#valid.drop(['label'],axis=1, inplace=True)

In [ ]:
#Image.open(train['img_path'][0])

In [ ]:
valid.head()

,id,img_path,label
0,329,./label_0/329.jpg,0
1,110,/content/dataset/flip/110.jpg,1
2,84,./미용실_광고_이미지_청수/image_ad16.jpg,1
3,942,./label_0/942.jpg,0
4,137,./이대미용실추가/137.jpg,1


In [ ]:
len(train), len(valid)

(2837, 116)

In [ ]:
# 현재 
# 2759 => 블로그, 인스타 
# 101 => 네이버 크롤링한 valid 
# 93 => 네이버 크롤링 test
 
# 어떻게?
# 2963(total), 204(네이버지도)
# 네이버지도를 train(150) valid(50)으로 변경  

# train-valid 바꾼다음 

'''
1. aug 적용시켜서 하는거 
2. enb3, vgg16 -> epoch 변경
3. cutmix 적용 (code)
4. data 추가 / Test data 변경 
'''


In [ ]:
#train.head()

,id,img_path,label
0,837,./label_1/837.jpg,1
1,73,./label_0/73.jpg,0
2,351,./label_0/351.jpg,0
3,1066,./label_1/1066.jpg,1
4,531,./label_0/531.jpg,0


# Data Preprocessing

In [ ]:
def get_labels(df):
    return df.iloc[:,2:].values

In [ ]:
train_labels=get_labels(train)
valid_labels=get_labels(valid)

# CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            # 좌우 반전 
                            #A.HorizontalFlip(p=0.3),
                            # 상하 반전 
                            #A.VerticalFlip(p=0.3),
                            # 검은 박스 여러개
                            #A.CoarseDropout(p=1.0, max_holes=8, max_height=21, max_width=21, min_holes=8, min_height=8,
                            #                min_width=8, fill_value=(0,0,0), mask_fill_value=None),
                            # 밝기 대비 
                            #A.RandomBrightnessContrast(p=0.5),
                             
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

valid_dataset = CustomDataset(valid['img_path'].values, valid_labels, test_transform)
valid_loader = DataLoader(valid_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
for x,y in train_loader:
    print(f'Image shape : {x.shape}')
    print(f'Label shape : {y.shape}')
    break

Image shape : torch.Size([16, 3, 456, 456])
Label shape : torch.Size([16, 1])


# Model Define

In [ ]:
#!pip install efficientnet_pytorch
#from efficientnet_pytorch import EfficientNet

!pip install timm
import timm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 30.4 MB/s eta 0:00:00


In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=1):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b3(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        #self.model = timm.create_model('vgg16', pretrained=True, num_classes=1)

    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x
        #x = F.sigmoid(self.model(x))
        #return x
 

In [ ]:
def train(model, optimizer, train_loader, valid_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_valid_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _valid_loss, _valid_acc = validation(model, criterion, valid_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Valid Loss : [{_valid_loss:.5f}] Valid ACC : [{_valid_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_valid_acc)
            
        if best_valid_acc < _valid_acc:
            best_valid_acc = _valid_acc
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, valid_loader, device):
    model.eval()
    valid_loss = []
    valid_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(valid_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs >= 0.5
            batch_acc = (labels == preds).mean()
            
            valid_acc.append(batch_acc)
            valid_loss.append(loss.item())
        
        _valid_loss = np.mean(valid_loss)
        _valid_acc = np.mean(valid_acc)
    
    return _valid_loss, _valid_acc

# Run!

In [ ]:
model = BaseModel()
#model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, valid_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0/178 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.58133] Valid Loss : [0.60015] Valid ACC : [0.64062]


  0%|          | 0/178 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.21038] Valid Loss : [0.67337] Valid ACC : [0.71094]


  0%|          | 0/178 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.18202] Valid Loss : [0.85997] Valid ACC : [0.64844]


In [ ]:
torch.save(infer_model.state_dict(), '/content/drive/MyDrive/vgg16_epoch10.pt')

In [ ]:
# 모델 불러오기 
infer_model = BaseModel()
infer_model.load_state_dict(torch.load('/content/drive/MyDrive/review_model.pt'))
infer_model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth


  0%|          | 0.00/47.2M [00:00<?, ?B/s]

BaseModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
              (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Inference

In [ ]:
!unzip -qq "/content/drive/MyDrive/비타민 자율프로젝트(1조)/image_최종/test_img.zip"

In [ ]:
#test = pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/img_test.csv')
'''
폴더 이름들 다시 수정해줘야 오류가 발생 X 
'''
#test = pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/data_split/final_test.csv')
test = pd.read_csv('/content/drive/MyDrive/비타민 자율프로젝트(1조)/csv/new_test.csv')
test.drop(['content'],axis=1,inplace=True)
test = test.iloc[:,:]
test.reset_index(inplace=True, drop=True)
test

,id,img_path,label
0,1,./test_img/1.jpg,0.0
1,2,./test_img/2.jpg,0.0
2,3,./test_img/3.jpg,0.0
3,4,./test_img/4.jpg,0.0
4,5,./test_img/5.jpg,0.0
...,...,...,...
99,101,./test_img/101.jpg,1.0
100,102,./test_img/102.jpg,0.0
101,103,./test_img/103.jpg,0.0
102,104,./test_img/104.jpg,0.0


In [ ]:
'''
test_onehot = pd.get_dummies(test['label'])
test = pd.concat([test, test_onehot], axis=1)
test.drop(['label'],axis=1, inplace=True)
test
'''

,img_path,id,0,1
0,./미용실_비광고_이미지_은비/35.jpg,18,1,0
1,./미용실_광고_이미지_유정/19.jpg,32,0,1
2,./미용실_비광고_이미지_은비/19.jpg,34,1,0
3,./미용실_광고_이미지_청수/image_ad41.jpg,71,0,1
4,./미용실_비광고_이미지_은비/17.jpg,9,1,0
...,...,...,...,...
97,./미용실_광고_이미지_유정/35.jpeg,29,0,1
98,./미용실_광고_이미지_유정/27.jpg,24,0,1
99,./미용실_광고_이미지_청수/image_ad50.jpg,78,0,1
100,./미용실_광고_이미지_유정/39.jpeg,25,0,1


In [ ]:
#test[0] = 0
#test[1] = 0 

In [ ]:
temp = test.copy()
#temp.drop([0,1], axis=1, inplace=True)
temp.drop(['label'], axis=1, inplace=True)
temp

,id,img_path
0,1,./test_img/1.jpg
1,2,./test_img/2.jpg
2,3,./test_img/3.jpg
3,4,./test_img/4.jpg
4,5,./test_img/5.jpg
...,...,...
99,101,./test_img/101.jpg
100,102,./test_img/102.jpg
101,103,./test_img/103.jpg
102,104,./test_img/104.jpg


In [ ]:
Image.open(temp['img_path'][0])

In [ ]:
test_dataset = CustomDataset(temp['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            #preds = probs >= 0.75
            preds = probs >= 0.5 # 아니면 0.75일수도 
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
preds[0:3]

[[1, 0], [1, 0], [1, 0]]

# Submission

In [ ]:
submit = test.copy()

submit.iloc[:,2:] = preds
submit.head()

,id,img_path,label
0,1,./test_img/1.jpg,1
1,2,./test_img/2.jpg,1
2,3,./test_img/3.jpg,1
3,4,./test_img/4.jpg,0
4,5,./test_img/5.jpg,0


In [ ]:
test_pred = submit.iloc[:,2:]
test_real = test.iloc[:,2:]

In [ ]:
submit

In [ ]:
# 전체 = 0.56, 성우 = 0.88, 청수 = 0.28, 유정(~50) = 0.24 , 은비 = 0.849

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_real, test_pred))

0.6346153846153846


In [ ]:
submit['real_label'] = test['label']
submit.head()

,id,img_path,label,real_label
0,1,./test_img/1.jpg,1,0.0
1,2,./test_img/2.jpg,1,0.0
2,3,./test_img/3.jpg,1,0.0
3,4,./test_img/4.jpg,0,0.0
4,5,./test_img/5.jpg,1,0.0


In [ ]:
submit.to_csv('vgg_0.5.csv', index=False)

# 확률 출력

In [ ]:
def inference_ensemble(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs
            preds = preds.astype(float)
            predictions += preds.tolist()
    return predictions

In [ ]:
prob_preds=inference_ensemble(infer_model, test_loader, device)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
prob_preds[0]

[0.12671352922916412]

In [ ]:
test

,img_path,id,0,1
0,./미용실_비광고_이미지_은비/35.jpg,18,1,0
1,./미용실_광고_이미지_유정/19.jpg,32,0,1
2,./미용실_비광고_이미지_은비/19.jpg,34,1,0
3,./미용실_광고_이미지_청수/image_ad41.jpg,71,0,1
4,./미용실_비광고_이미지_은비/17.jpg,9,1,0
...,...,...,...,...
97,./미용실_광고_이미지_유정/35.jpeg,29,0,1
98,./미용실_광고_이미지_유정/27.jpg,24,0,1
99,./미용실_광고_이미지_청수/image_ad50.jpg,78,0,1
100,./미용실_광고_이미지_유정/39.jpeg,25,0,1


In [ ]:
submit_probs = test.copy()
submit_probs.iloc[:,2:] = prob_preds
submit_probs['real_label'] = test['label']
#submit_probs.iloc[:,3:] = prob_preds
submit_probs.head()

,id,img_path,label,real_label
0,1,./test_img/1.jpg,0.981246,0.0
1,2,./test_img/2.jpg,0.999084,0.0
2,3,./test_img/3.jpg,0.745808,0.0
3,4,./test_img/4.jpg,0.008523,0.0
4,5,./test_img/5.jpg,0.002006,0.0


In [ ]:
submit_probs.to_csv('probs_enb3_epoch5_0.5.csv', encoding='utf-8-sig',index = False)